# EN PROCESO

In [1]:
%load_ext autoreload
%autoreload 2
%env MKL_NUM_THREADS=1

env: MKL_NUM_THREADS=1


In [2]:
import sys
sys.path.append("../libs")
from vvv_utils import parse_metadata, parse_light_curve_data, vvv_path, get_train_test_ids

df_meta = parse_metadata(experiment="ALL", merge_subclasses=True)
# train_idx, test_idx = get_train_test_ids(df_meta)

88454 light curve metadata collected


In [3]:
from joblib import load, dump
from os.path import join

with open(join(vvv_path, "features/features_fats_demo.pkl"), "rb") as f:
    features = load(f)
    
features.head(5)

,CAR_sigma,CAR_mean,Meanvariance,Mean,PercentDifferenceFluxPercentile,PercentAmplitude,Skew,AndersonDarling,Std,Rcs,...,Eta_e,Period_fit_v2,PeriodPowerRate,Psi_CS_v2,Psi_eta_v2,StructureFunction_index_21,Pvar,StructureFunction_index_31,ExcessVar,IAR_phi
oid,,,,,,,,,,,,,,,,,,,,,
b221_201_22183,0.029854,0.129690,0.004019,12.969043,0.011624,0.007798,0.299589,1.000000,0.052119,0.129087,...,0.085593,0.199446,0.006845,0.269202,0.174647,1.410958,1.0,1.567173,0.000015,0.395942
b221_205_41463,0.014141,0.122816,0.003381,12.281559,0.011656,0.008952,0.319367,0.996356,0.041528,0.160954,...,0.037231,0.088535,0.003884,0.239514,0.603361,2.110024,1.0,3.264560,0.000009,0.657367
b232_614_24529,0.012187,0.126573,0.002282,12.657314,0.007116,0.006595,-0.595523,0.997524,0.028881,0.152810,...,0.316498,0.010538,0.009164,0.198155,0.860321,1.734619,1.0,2.415084,0.000004,0.570542
b233_201_30278,0.012476,0.130966,0.002302,13.096551,0.007336,0.007901,0.331045,0.926431,0.030146,0.136696,...,0.024233,0.005611,0.201240,0.220840,1.208544,1.551493,1.0,1.957971,0.000004,0.283583
b233_201_16631,0.027895,3.682501,0.003355,12.692367,0.009962,0.011822,0.456372,0.999898,0.042577,0.144025,...,0.165085,0.139859,0.004803,0.278597,0.372389,1.692609,1.0,2.254031,0.000010,0.769216


In [8]:
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

In [9]:
labels = np.array(df_meta.loc[features.index]["label"])
np.unique(labels)

array(['binary'], dtype=object)

In [ ]:
parameters = {'rf__criterion':('gini', 'entropy'), 'rf__n_estimators':[50, 500, 5000], 'rf__max_depth': [None, 10],
              'aug__sampling_strategy': ('majority', 'not minority', 'all'), 'aug__n_neighbors': [3, 5, 10],
              'rf__class_weight': (None, 'balanced', 'balanced_subsample')}
aug = ADASYN()
rf = RandomForestClassifier()
pipe = Pipeline([('aug', aug), ('rf', rf)])

cv = GridSearchCV(pipe, parameters, cv=4, verbose=2, n_jobs=20, scoring='f1_macro')
cv.fit(features.values, train_labels[:])